# Construire un corpus des scripts de la série *Kaamelott*

## Récupérer la liste des scripts

Scripts disponibles sur le site collaboratif Hypnoweb :  
[https://kaamelott.hypnoweb.net/kaamelott/guide.119.2/](https://kaamelott.hypnoweb.net/kaamelott/guide.119.2/)

**Procédure :**
1. lire la page
2. sélectionner dans le code HTML les liens vers les épisodes
3. écrire chaque lien dans un fichier

**1e étape :** lire la page

In [ ]:
#!/usr/bin/python
#-*- coding: utf-8 -*-

# Modules to import
import scrape.utils as scrape

# Main procedure
if __name__ == '__main__':
    url = 'https://kaamelott.hypnoweb.net/kaamelott/guide.119.2/'
    html = scrape.get_html_from_url(url, 'ISO-8859-1')

**2e étape :** sélectionner les liens vers les épisodes

In [ ]:
episodes = scrape.parse_html_by_class(html, '.tabs-content .content tr td a')
links = [
    episode.get('href')
    for episode in episodes
]

**3e étape :** écrire chaque lien dans un fichier

In [ ]:
with open('./data/kaamelott/episodes.txt', 'w') as file:
    [ file.write(link + '\n') for link in links ]

Autre possibilité : écriture du fichier grâce au mode *append* :

In [ ]:
# List of episodes
episodes = scrape.parse_html_by_class(html, '.tabs-content .content tr td a')

# For each episode
for episode in episodes:

    # File descriptor to destination file
    with open('./data/kaamelott/episodes.txt', 'a') as file:

        # What's the link?
        link = episode.get('href')

        # Write the file
        file.write(link + '\n')

## Extraire les scripts

**Procédure :**
1. lire le fichier `episodes.txt` ligne par ligne
2. reconstruire les urls
3. lire chaque url
4. extraire les scripts
5. écrire un fichier texte par script

**1e étape :** lire le fichier `episodes.txt`

In [ ]:
# File descriptor
with open('./data/kaamelott/episodes.txt') as file:
    # A list of lines
    lines = file.readlines()

**2e étape :** reconstruire les URL pour passer d’un chemin relatif vers une adresse absolue

Exemple :
- ligne du fichier :  
`/kaamelott/episode.119.2/S01E01-heat-2956.html`  
- adresse absolue :  
`https://kaamelott.hypnoweb.net/kaamelott/episode.119.2/S01E01-heat-2956.html`

In [ ]:
for line in lines:
    url = f"https://kaamelott.hypnoweb.net{line}"

**3e étape :** lire chaque URL

In [ ]:
for line in lines:
    url = f"https://kaamelott.hypnoweb.net{line}"
    html = scrape.get_html_from_url(url, "ISO-8859-1")

**4e étape :** extraire les scripts

In [ ]:
for line in lines:
    url = f"https://kaamelott.hypnoweb.net{line}"
    html = scrape.get_html_from_url(url, "ISO-8859-1")
    # script_vo <div> describes the scripts
    scripts = scrape.parse_html_by_class(html, "#script_vo div")

**Dernière étape :** écrire un fichier texte par script

Nom de chaque fichier sous la forme `SaisonEpisode-titre-episode.txt`.

Par exemple, pour la ligne :  
`/kaamelott/episode.119.2/S01E09-le-maitre-d-armes-2964.html`

le nom du fichier qui contient le script sera : `S01E09-le-maitre-d-armes.txt`

Une expression régulière permettra d’isoler le titre de l’épisode dans chaque URL. Pour utiliser une regex avec Python, il faut importer le module `re` :

In [ ]:
import re

Les expressions régulières fonctionnent par application d’un motif sur une chaîne de caractère. Le résultat renvoyé sert à formater le nom du fichier de destination.

In [ ]:
# Pattern:
# - brackets capture the pattern
# - ?P<episode> creates a named group 'episode' with content of the regex
pattern = '(?P<episode>[0-9ES]{6}-[a-z0-9-]*)-[0-9]*\.html'

for line in lines:
    url = f"https://kaamelott.hypnoweb.net{line}"
    html = scrape.get_html_from_url(url, "ISO-8859-1")
    scripts = scrape.parse_html_by_class(html, "#script_vo div")
    # match method captures the pattern in each line in a variable called 'm'
    m = re.match(pattern, line.replace("/kaamelott/episode.119.2/", ""))

Il ne reste plus qu'à écrire le texte des scripts dans chaque fichier :

In [ ]:
pattern = '(?P<episode>[0-9ES]{6}-[a-z0-9-]*)-[0-9]*\.html'

for line in lines:
    url = f"https://kaamelott.hypnoweb.net{line}"
    html = scrape.get_html_from_url(url, "ISO-8859-1")
    scripts = scrape.parse_html_by_class(html, "#script_vo div")
    m = re.match(pattern, line.replace("/kaamelott/episode.119.2/", ""))

    # 'group' method on 'm' access the content of the named group 'episode'
    with open(f"./data/kaamelott/raw/{m.group('episode')}.txt", 'w') as file:
        for text in scripts[0].stripped_strings:
            file.write(text)
            file.write('\n')

Enfin, comme tous les épisodes n’ont pas été retranscrits, on ajoute une condition à l’écriture du fichier :

In [ ]:
pattern = '(?P<episode>[0-9ES]{6}-[a-z0-9-]*)-[0-9]*\.html'

for line in lines:

    url = f"https://kaamelott.hypnoweb.net{line}"
    html = scrape.get_html_from_url(url, "ISO-8859-1")
    scripts = scrape.parse_html_by_class(html, "#script_vo div")
    m = re.match(pattern, line.replace("/kaamelott/episode.119.2/", ""))

    # If there is a content…
    if len(scripts) != 0:
        with open(f"./data/kaamelott/raw/{m.group('episode')}.txt", 'w') as file:
            for text in scripts[0].stripped_strings:
                file.write(text)
                file.write('\n')